<span style='font-size:35px;'><span style='color:#860DFF;'>E</span><span style='color:#8311FF;'>r</span><span style='color:#8015FF;'>r</span><span style='color:#7D19FF;'>o</span><span style='color:#7B1DFF;'>r</span> <span style='color:#7525FF;'>4</span><span style='color:#7329FF;'>0</span><span style='color:#702EFF;'>4</span> <span style='color:#6B36FF;'>3</span><span style='color:#683AFF;'>7</span><span style='color:#653EFF;'>3</span><span style='color:#6342FF;'>5</span><span style='color:#6046FF;'>C</span> <span style='color:#5B4FFF;'>C</span><span style='color:#5853FF;'>o</span><span style='color:#5557FF;'>d</span><span style='color:#525BFF;'>e</span> <span style='color:#4D63FF;'>E</span><span style='color:#4A67FF;'>x</span><span style='color:#486BFF;'>p</span><span style='color:#4570FF;'>l</span><span style='color:#4274FF;'>a</span><span style='color:#4078FF;'>n</span><span style='color:#3D7CFF;'>a</span><span style='color:#3A80FF;'>t</span><span style='color:#3884FF;'>i</span><span style='color:#3588FF;'>o</span><span style='color:#328CFF;'>n</span></span>

<hr></hr>

<span style='font-size:27px;'><span style='color:#860DFF;'>O</span><span style='color:#7B1DFF;'>v</span><span style='color:#702EFF;'>e</span><span style='color:#653EFF;'>r</span><span style='color:#5B4FFF;'>v</span><span style='color:#505FFF;'>i</span><span style='color:#4570FF;'>e</span><span style='color:#3A80FF;'>w</span></span>

There are a few interesting bits of code I would like to explain. I'll divide them into three categories:
<ol>
    <li>X-Drive Logic,</li>
    <li>Shooting (flywheel, auto aiming), and</li>
    <li>Auton (wheel odometry)</li>
</ol>

<hr></hr>

<span style='font-size:27px;'><span style='color:#860DFF;'>X</span><span style='color:#7F17FF;'>-</span><span style='color:#7821FF;'>D</span><span style='color:#722BFF;'>r</span><span style='color:#6B35FF;'>i</span><span style='color:#643FFF;'>v</span><span style='color:#5E49FF;'>e</span> <span style='color:#515EFF;'>L</span><span style='color:#4A68FF;'>o</span><span style='color:#4372FF;'>g</span><span style='color:#3D7CFF;'>i</span><span style='color:#3686FF;'>c</span></span>

Our robot uses an x-drive drivetrain, which, though not incredibly common, already has pretty standardized code. They can go forward, backwards, and strafe side-to-side. Our x-drive is unique because it is field oriented; it's the closest you can get to running a swerve drive on a vex bot. Field oriented driving means that, no matter what direction the robot is facing, it goes forward relative to you/the field, not itself. So if the robot is facing left, and you tell it to go forward, it'll strafe to its right.

<img src="https://github.com/elee012345/vex-spin-up/blob/main/code-explanations/images/field%20oriented.png" alt="Field Oriented Driving Diagram">

Code is below
```cpp
double headingRadians = Inertial2.heading() * 3.14159/180;
double yPos = con1.Axis3.position(pct);
double xPos = con1.Axis4.position(pct);
double sineHeading = sin(headingRadians);
double cosHeading = cos(headingRadians);
// rotate the controller x/y coordinates by the negative of the robot heading
// robot is facing 30 degrees and we tell it to go forward, then it needs to strafe -30 degrees
double rotatedYPos = xPos * sineHeading + yPos * cosHeading;
double rotatedXPos = xPos * cosHeading - yPos * sineHeading;
```

<hr></hr>

ur mom

Code is below
```cpp
// Vex internal PID means this code isn't turned correctly for our flywheel
vex::motor::spin(directionType::fwd, 100, velocityUnits::pct)

// Controlling the flywheel motors with voltage bypasses the built-in Vex PID
vex::motor::spin(directionType::fwd, 12, voltageUnits::volt)
```